In [ ]:
# ! pip install -U weaviate-client pandas

In [16]:
# import weaviate
# from weaviate.classes.init import Auth 

# # Best practice: store your credentials in environment variables
# weaviate_url = 'https://dbvzpt0resxqjgufx5ing.c0.us-west3.gcp.weaviate.cloud'
# weaviate_api_key = ''

# # Connect to Weaviate Cloud
# client = weaviate.connect_to_weaviate_cloud(
#     cluster_url=weaviate_url, 
#     auth_credentials=Auth.api_key(weaviate_api_key),
# )

# print(client.is_ready()) 

True


In [29]:
import weaviate
 
client = weaviate.connect_to_local()

print(client.is_ready())

True


/mnt/ceph/develop/jiawei/weaviate-python-client/weaviate/warnings.py:133: DeprecationWarning: Dep005: You are using weaviate-client version 0.1.dev3117+gae1bb03. The latest version is 4.10.2.
            Consider upgrading to the latest version. See https://weaviate.io/developers/weaviate/client-libraries/python for details.
  warnings.warn(


In [31]:
client.collections.delete("OpenVidLateContext_jina_v2_zh") 


In [32]:
from weaviate.classes.config import Configure, Property, DataType, VectorDistances
 
import weaviate.classes as wvc
import weaviate.classes.config as wvcc
client.collections.create(
    "OpenVidLateContext_jina_v2_zh",
    
    vectorizer_config=wvc.config.Configure.Vectorizer.none(),
  
)

In [33]:
 
import os
import requests

file_path = "OpenVid-1M.csv"

# 检查文件是否存在，如果不存在则下载
if not os.path.exists(file_path):
    print("文件不存在，正在下载...")
    response = requests.get(
        "https://huggingface.co/datasets/nkp37/OpenVid-1M/resolve/main/data/train/OpenVid-1M.csv"
    )
    # 保存文件
    with open(file_path, "wb") as f:
        f.write(response.content)
    print("文件下载完成！")
else:
    print("文件已存在，无需下载。")

# 确认文件状态
file_exists = os.path.exists(file_path)
file_exists
 

文件已存在，无需下载。


True

### 操作方式

以下是将文本分块为句子的一些通用函数，以及后期分块背后的大部分操作。

后期分块与我们在简单分块的文本上看到的块完全相同，但块嵌入取自标记嵌入的池化，而不是独立嵌入的块。

In [34]:
def chunk_by_sentences(input_text: str, tokenizer: callable, mark_text: str='.'):
    """
    Split the input text into sentences using the tokenizer
    :param input_text: The text snippet to split into sentences
    :param tokenizer: The tokenizer to use
    :return: A tuple containing the list of text chunks and their corresponding token spans
    """
    inputs = tokenizer(input_text, return_tensors='pt', return_offsets_mapping=True)
    punctuation_mark_id = tokenizer.convert_tokens_to_ids(mark_text)
    sep_id = tokenizer.convert_tokens_to_ids('[SEP]')
    token_offsets = inputs['offset_mapping'][0]
    token_ids = inputs['input_ids'][0]
    chunk_positions = [
        (i, int(start + 1))
        for i, (token_id, (start, end)) in enumerate(zip(token_ids, token_offsets))
        if token_id == punctuation_mark_id
        and (
            token_offsets[i + 1][0] - token_offsets[i][1] > 0
            or token_ids[i + 1] == sep_id
        )
    ]
    chunks = [
        input_text[x[1] : y[1]]
        for x, y in zip([(1, 0)] + chunk_positions[:-1], chunk_positions)
    ]
    span_annotations = [
        (x[0], y[0]) for (x, y) in zip([(1, 0)] + chunk_positions[:-1], chunk_positions)
    ]
    return chunks, span_annotations
 
  
def late_chunking(
    model_output: 'BatchEncoding', span_annotation: list, max_length=None
):
    token_embeddings = model_output[0]
    outputs = []
    for embeddings, annotations in zip(token_embeddings, span_annotation):
        if (
            max_length is not None
        ):  # remove annotations which go bejond the max-length of the model
            annotations = [
                (start, min(end, max_length - 1))
                for (start, end) in annotations
                if start < (max_length - 1)
            ]
        pooled_embeddings = [
            embeddings[start:end].sum(dim=0) / (end - start)
            for start, end in annotations
            if (end - start) >= 1
        ]
        pooled_embeddings = [
            embedding.detach().cpu().numpy() for embedding in pooled_embeddings
        ]
        outputs.append(pooled_embeddings)

    return outputs


### Import into Weaviate

现在让我们使用视频描述数据集。我们稍后将使用late chunking/chunking查询此文本。
此处我们使用分批次导入的方式，对每条数据的描述文本上下文embedding，然后将其导入 

“OpenVidLateContext_jina_v2_zh”的 Weaviate 集合。

现在，从 Huggingface 加载 jinaai/jina-embeddings-v2-base-zh 模型。 这个模型支持中英双语的文本向量模型，
这对于后期分块很重要，因为我们希望对大型文档进行编码，然后再将它们分开。

In [35]:

import transformers
from transformers import AutoModel
from transformers import AutoTokenizer

# load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained('/mnt/ceph/develop/jiawei/model_checkpoint/jina-embeddings-v2-base-zh', trust_remote_code=True)
model = AutoModel.from_pretrained('/mnt/ceph/develop/jiawei/model_checkpoint/jina-embeddings-v2-base-zh', trust_remote_code=True).to(device='cuda:0') 

我们调用我们之前定义的函数：这里将每行对应一个上下文，我们希望对embedding后的token带有整体的信息，这里使用mark_text为`.`分割
后，以获得块的起点和终点。
然后嵌入整个文档。然后执行late chunking步骤 - 对与每个块相对应的所有标记嵌入取平均值（基于块的起点/终点）
这些形成我们的块embedding。

In [ ]:
import pandas as pd

import transformers
from transformers import AutoModel
from transformers import AutoTokenizer
# Settings for displaying the import progress
counter = 0
interval = 200  # print progress every this many records; should be bigger than the batch_size
collection = client.collections.get("OpenVidLateContext_jina_v2_zh")

print("JSON streaming, to avoid running out of memory on large files...")
  

# 分块读取 CSV 文件
chunk_size = 10000   # 每次读取10000行
for idx, chunk in enumerate(pd.read_csv(file_path, chunksize=chunk_size)):
    print(f"Processing chunk {idx + 1}...")
    for _, row in chunk.iterrows():
        if idx<1:
            continue
        # 读取每一行的视频信息和标题等
        video_filename = row['video']
        caption = row['caption']
        chunks, span_annotations = chunk_by_sentences(caption, tokenizer) 
        inputs = tokenizer(caption, return_tensors='pt').to(device='cuda:0')
        model_output = model(**inputs)
        chunk_embeddings = late_chunking(model_output, [span_annotations])[0] 
    
        # add data with manual embeddings
        data = []
        for i in range(len(chunks)):
            data.append(wvc.data.DataObject(
          
                    properties = {
                        "Video": video_filename,
                        "Caption": caption,
                        "content": chunks[i]
                    },
                    vector = chunk_embeddings[i].tolist()
            )
        )
        # 向量数据库插入数据
        collection.data.insert_many(data);
   

        counter += 1

        # 每处理200条数据输出一次进度
        if counter % interval == 0:
            print(f"Imported {counter} records...")
            
# 完成后输出总进度
print(f"Finished importing {counter} records.")

JSON streaming, to avoid running out of memory on large files...
Processing chunk 1...
Processing chunk 2...


In [37]:

collection_shards = collection.config.get_shards()
print(collection_shards)

[_ShardStatus(name='WAjUMOUHNYlr', status='READY', vector_queue_size=0)]


In [38]:
for item in collection.iterator():
    print(item.uuid, item.properties)
    break

000094e9-73ac-49a2-b01c-0e61ded89d0d {'content': ' The style of the video is a close-up, real-time shot, focusing on the person and the computer setup.', 'caption': "The video shows a person working on a computer setup. The person is wearing a black hoodie with an orange logo on the front. They are holding a red handle, possibly a tool, and are interacting with the computer setup. The setup includes various electronic components such as a motherboard, a power supply, and a fan. The person appears to be in the process of assembling or repairing the computer. The style of the video is a close-up, real-time shot, focusing on the person and the computer setup. The video captures the details of the electronic components and the person's actions, providing a clear view of the process.", 'video': '-HO6Ys3PPIw_21_48to256.mp4'}
